# CSCA 5622 Final Project - DOROTHEA Dataset Supervised Learning Model

#### The DOROTHEA dataset (downloaded from UCI https://archive.ics.uci.edu/dataset/169/dorothea) contains data used for drug discovery. As possible drugs are considered, the most important consideration is if they bind to the desired receptor or not. This dataset maps potential drug substances to active (+1) or inactive (-1) depending on their binding activity to the desired receptor.

##### This dataset is altered to include probe features with no predictive power as a means to test feature selection, so this project will emphasize feature selection as a main part of the modeling.

In [1]:
import pandas as pd
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as sfs